# Problem 935 - Unlucky Primes
<p>We define the <i>unlucky prime</i> of a number $n$, denoted $u(n)$, as the smallest prime number $p$ such that the remainder of $n$ divided by $p$ (i.e. $n \bmod p$) is not a multiple of seven.<br>
For example, $u(14) = 3$, $u(147) = 2$ and $u(1470) = 13$.</p>

<p>Let $U(N)$ be the sum $\sum_{n = 1}^N u(n)$.<br>
You are given $U(1470) = 4293$.</p>

<p>Find $U(10^{17})$.</p>

### Solution.

https://en.wikipedia.org/wiki/Chinese_remainder_theorem

In [50]:
from functools import lru_cache, reduce
from collections import defaultdict
from math import ceil, prod, gcd
import matplotlib.pyplot as plt
from tqdm import tqdm
import timeit

In [20]:
def generate_primes(N):
    if N < 2:
        return []

    primes = [True] * (N + 1)
    primes[0] = primes[1] = False
    
    for i in range(2, int(N**0.5) + 1):
        if primes[i]:
            for j in range(i * i, N + 1, i):
                primes[j] = False
    
    return [i for i, is_prime in enumerate(primes) if is_prime]

primes = generate_primes(10000)

In [44]:
from math import gcd

@lru_cache
def mod_inverse(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        a, m = m, a % m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1


def solve_crt(a, m):
    a1, a2 = a
    m1, m2 = m
    
    M = m1 * m2
    M1, M2 = M // m1, M // m2
    inv1, inv2 = mod_inverse(M1, m1), mod_inverse(M2, m2)
    
    x = (a1 * M1 * inv1 + a2 * M2 * inv2) % M
    return x % M


In [45]:
def U(N):
    ans = 0
    queue = [(0, 2, 2), (1, 2, 2)]
    
    i = 1
    total = 0
    while total != N:
        next_prime = primes[i]
        new_queue = []

        for q in queue:
            r, p, product = q

            if r <= N:
                if r % p % 7 != 0:
                    ans += N//product * p
                    total += N//product

                    if r <= N % product:
                        ans += p
                        total += 1
                
                else:
                    for j in range(1, next_prime+1):
                        new_queue.append((solve_crt([r, j], [product, next_prime]), next_prime, product * next_prime))

        i += 1
        queue = new_queue


    return ans

In [57]:
round(timeit.timeit("U(10000)", globals=globals(), number=1),5)

0.0001

In [59]:
for i in range(1, 18):
    print(i, U(10**i))

1 27
2 288
3 2914
4 29202
5 292128
6 2921350
7 29213782
8 292137790
9 2921378026
10 29213780802
11 292137809368
12 2921378094786
13 29213780949138
14 292137809489660
15 2921378094903954
16 29213780949043308
17 292137809490441370


ANS: $U(10^{17}) = 292137809490441370$.

## Simulation.

In [12]:
from sympy import primerange

def u(n):
    for p in primerange(1, 2*n+1):
        if (n % p) % 7 != 0:
            return p

def U_sim(N):
    ans = 0
    for n in range(1, N+1):
        ans += u(n)
    return ans

In [13]:
freq = defaultdict(int)

N = 10**3
for n in tqdm(range(1, N+1)):
    freq[u(n)] += 1

U_sim(n), freq

100%|██████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 325013.87it/s]


(2914, defaultdict(int, {2: 500, 3: 334, 5: 133, 7: 29, 11: 4}))